<table style="width: 100%; border-style: none;">
<tr style="border-style: none">
<td style="border-style: none; width: 1%; font-size: 16px">Institut f&uuml;r Theoretische Physik<br /> Universit&auml;t zu K&ouml;ln</td>
<td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Prof. Dr. Simon Trebst<br />Peter Br&ouml;cker</td>
</tr>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Computerphysik</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Vorlesung &mdash; Programmiertechniken 11
</h1>
<hr>
<h3 style="font-weight:bold; text-align: center; margin: 0px; padding:0px; margin-bottom: 20px;">Sommersemester 2016</h3>

**Website:** [http://www.thp.uni-koeln.de/trebst/Lectures/2016-CompPhys.shtml](http://www.thp.uni-koeln.de/trebst/Lectures/2016-CompPhys.shtml)

# Monte Carlo Simulations des zwei-dimensionalen Ising-Modells

### Initialisierungen


In [1]:
type parameters
    L::Int
    beta::Float64
    addition_prob::Float64
    
    parameters() = new(32, 2.5, 0.5)
end

## Lokaler Update

In [12]:
function local_sweep!(p::parameters, spins::Array{Int, 1}; no_spins=p.L*p.L)
    # get parameter
    L    = p.L
    beta = p.beta
    
    # flip spins
    # ein sweep durch das Gitter
    for i in 1:L*L
        # random spin
        r = convert(Int, ceil(rand()*L))  # row
        c = convert(Int, floor(rand()*L)) # column
        
        idx = r + c * L
        flip_spin = spins[idx]
        
        # energy difference
        delta_E = 0
        for s in [    mod1(r+1, L) + c*L,   # get right neighbour modulo
                      mod1(r-1+L, L) + c*L,
                      r + mod1(c+1, L) * L,
                      r + mod1(c-1+L, L) * L]
            delta_E += flip_spin * spins
        end
        
        # acceptance probability
        probability = exp(-beta*delta_E)
        
        # Metropolis update
        if rand() < probability
            # flip spin
            spins[idx] *= -1
        end
    end
end

local_sweep! (generic function with 1 method)

## Cluster-Update / Wolff-Algorithmus

In [ ]:
function wolff_sweep!(p::parameters, spins::Array{Int, 1})
    spins_flipped = 0
    L = p.L
    
    # random spin
    r = convert(Int, ceil(rand()*p.L))
    c = convert(Int, floor(rand()*p.L))

    initial_idx = r + c * p.L
    initial_spin = spins[initial_idx]

    # cluster list
    to_flip = [(r, c)]

    while length(to_flip) != 0
        (r, c) = to_flip[1]
        deleteat!(to_flip, 1)

        # flip spin in cluster and look out whether to accept neighbors to cluster as well
        if spins[r + c * p.L] == initial_spin
            spins[r + c * p.L] *= -1
            spins_flipped += 1
            for s in [(mod1(r + 1, L) + c * L, (mod1(r + 1, L), c))
                    , (mod1(r - 1 + L, L) + c * L, (mod1(r - 1 + L, L), c))
                    , (r + mod(c + 1, L) * L, (r, mod(c + 1, L)))
                    , (r + mod(c - 1 + L, L) * L, (r, mod(c - 1 + L, L)))]
                if spins[s[1]] == initial_spin
                    if rand() < p.addition_prob
                        push!(to_flip, s[2])
                    end
                end
            end
        end
    end
end

## Hauptprogramm

In [9]:
using PyPlot

#### Spin-Konfiguration

In [7]:
p = parameters()
L = 16
p.L = L
spins = rand([-1, 1], L^2);

# How many spin configurations do we want to sample for a given temperature?
samples = 100;

#### Temperaturen

kritische Temperatur (Onsager Lösung)

$\beta_c = \ln(1+\sqrt{2})/2 = 0.44068679350977...$

Die Boltzmann-Konstante wird hier auf $1$ gesetzt, da es keine Naturkonstante ist.

In [ ]:
# betas = [0.2, 0.3, 0.35, collect(0.4:0.01:0.5)..., 0.55, 0.6, 0.7, 0.8, 0.9, collect(1:10)...]
# betas = [0.8];

#### Observablen

Magnetisierung 

$ M = \sum_i \sigma_i $

Suszeptibilität

$ \chi = \left(\frac{\partial \langle M\rangle}{\partial H}\right)_T = \frac{1}{k_B\cdot T}\cdot\left( \langle M^2 \rangle - \langle M \rangle^2 \right) $

In [ ]:
xs = [] # T
magnetizations = []
susceptibilities = [];

#### graphische Ausgabe

In [14]:
# composite figure setup
pygui(true)
f=figure(figsize=(18, 9))

# spin configuration
ax_img = subplot2grid((2, 3), (0, 0), rowspan=2)
image = imshow(reshape(spins, p.L, p.L), interpolation="nearest", cmap="gist_gray", vmin=-1, vmax=1)
axis("off")

# # magnetization plot
# ax_mag = subplot2grid((2, 3), (0, 1), colspan=2)
# mag_plot = ax_mag[:plot]([], [])[1]
# mag_pntr = ax_mag[:plot]([], [], "o", markeredgecolor="none")[1]
# xlim([minimum(1./betas), maximum(1./betas)])
# ylim([-0.05, 1.05])
# xlabel("Temperatur")
# ylabel("Magnetisierung")

# # susceptibility plot
# ax_susc = subplot2grid((2, 3), (1, 1), colspan=2)
# susc_plot = ax_susc[:plot]([], [])[1]
# susc_pntr = ax_susc[:plot]([], [], "o", markeredgecolor="none")[1]
# xlim([minimum(1./betas), maximum(1./betas)])
# ylim([-1, L]);
# xlabel("Temperatur")
# ylabel("Suszeptibilität")

(-0.5,15.5,15.5,-0.5)

#### Temperatur-Scan

In [15]:
# betas = [0.2, 0.3, 0.35, collect(0.4:0.01:0.5)..., 0.55, 0.6, 0.7, 0.8, 0.9, collect(1:10)...]
betas = [0.0];

for beta in betas
    p.beta = beta
    p.addition_prob = 1.0 - exp(-2. * p.beta)
    
    magnetization   = 0.   # M
    magnetization_2 = 0. # M^2
    
    # Monte Carlo Simulation
    for sweep in 1:samples
        
        # lokaler Update
        local_sweep!(p, spins)
        image[:set_data](reshape(spins, p.L, p.L))
        sleep(0.0001)

        
#         # nicht-lokaler Update
#         wolff_sweep!(p, spins)
#         if mod(sweep, 51) == 0
#             image[:set_data](reshape(spins, p.L, p.L))
#             sleep(0.0001)
#         end
        
        magnetization   += abs(sum(spins))
        magnetization_2 += abs(sum(spins))^2        
    end
    
    push!(magnetizations, magnetization / L^2 / samples)

    susceptibility = beta * (magnetization_2/samples - (magnetization/samples)^2) /  L^2
    push!(susceptibilities, susceptibility)
 
    push!(xs, beta)
    
#     # figure updates
#     mag_plot[:set_data](1./xs, magnetizations)
#     mag_pntr[:set_data](1./xs[end], magnetizations[end])
    
#     susc_plot[:set_data](1./xs, susceptibilities)
#     susc_pntr[:set_data](1./xs[end], susceptibilities[end])
end

LoadError: LoadError: MethodError: `isless` has no method matching isless(::Float64, ::Array{Float64,1})
Closest candidates are:
  isless(::Float64, !Matched::Float64)
  isless(::AbstractFloat, !Matched::AbstractFloat)
  isless(::Real, !Matched::AbstractFloat)
  ...
while loading In[15], in expression starting on line 4